In [ ]:
import pandas as pd
import numpy as np
import unicodedata
import re
from datetime import datetime

def normalize_title(title):
    # Handle null values and non-string inputs
    # Returns empty string for NaN, None, or non-string values
    if pd.isna(title) or not isinstance(title, str):
        return ""
    
    # Unicode normalization (NFKD) handles special characters and accents
    # Converts "Café" to "Cafe", "naïve" to "naive", etc.
    # Also converts to lowercase for consistent comparison
    title = unicodedata.normalize('NFKD', title).lower()
    
    # Remove common English articles from the beginning
    # "The Matrix" -> "Matrix", "A Beautiful Mind" -> "Beautiful Mind"
    # Uses word boundary (\s+) to avoid removing partial matches
    title = re.sub(r'^(the|a|an)\s+', '', title)
    
    # Replace hyphens and underscores with spaces
    # "Spider-Man" -> "Spider Man", "Iron_Man" -> "Iron Man"
    # Prepares for space removal in final step
    title = re.sub(r'[-_]', ' ', title)
    
    # Remove parentheses/brackets and their contents
    # "Avatar (2009)" -> "Avatar", "The Matrix [Reloaded]" -> "The Matrix"
    # Handles both () and [] brackets with any content inside
    title = re.sub(r'\([^)]*\)|\[[^\]]*\]', '', title)
    
    # Replace colons and semicolons with spaces
    # "Mission: Impossible" -> "Mission Impossible"
    # "Fast & Furious; Tokyo Drift" -> "Fast & Furious Tokyo Drift"
    title = re.sub(r'[;:]', ' ', title)
    
    # Remove all punctuation except apostrophes and spaces
    # Keeps letters, numbers, spaces, and apostrophes only
    # "Fast & Furious!" -> "Fast   Furious", "Don't" -> "Don't"
    title = re.sub(r'[^\w\s\']', ' ', title)
    
    # Remove remaining apostrophes
    # "Don't Stop Me Now" -> "Dont Stop Me Now"
    # Ensures consistent matching for contractions
    title = re.sub(r'\'', '', title)
    
    # Remove ALL spaces and extra whitespace, then trim
    # "The   Dark    Knight" -> "thedarkknight"
    # Creates compact string for fuzzy matching algorithms
    title = re.sub(r'\s+', '', title).strip()
    
    return title

def clean_release_date(date_string):
    """
    Convert various date formats to PostgreSQL-compatible DATE format
    Examples: 
    - "January 1st" -> None (can't determine year)
    - "2021-05-15" -> "2021-05-15" 
    - "2021/05/15" -> "2021-05-15"
    - "December 25th" -> None (ambiguous without year)
    """
    if pd.isna(date_string) or not date_string:
        return None
    
    date_str = str(date_string).strip()
    
    # If it's already in YYYY-MM-DD format, keep it
    if re.match(r'^\d{4}-\d{2}-\d{2}$', date_str):
        return date_str
    
    # If it's in YYYY/MM/DD format, convert to YYYY-MM-DD
    if re.match(r'^\d{4}/\d{2}/\d{2}$', date_str):
        return date_str.replace('/', '-')
    
    # Handle MM/DD/YYYY format -> YYYY-MM-DD
    mm_dd_yyyy = re.match(r'^(\d{1,2})/(\d{1,2})/(\d{4})$', date_str)
    if mm_dd_yyyy:
        month, day, year = mm_dd_yyyy.groups()
        return f"{year}-{month.zfill(2)}-{day.zfill(2)}"
    
    # Handle DD/MM/YYYY format (European) -> YYYY-MM-DD
    # This is ambiguous with MM/DD/YYYY, but we'll assume MM/DD/YYYY is more common
    
    # Try to parse month names with ordinal indicators
    # "January 1st", "March 15th", etc. - but these lack year information
    month_patterns = {
        'january': '01', 'february': '02', 'march': '03', 'april': '04',
        'may': '05', 'june': '06', 'july': '07', 'august': '08',
        'september': '09', 'october': '10', 'november': '11', 'december': '12',
        'jan': '01', 'feb': '02', 'mar': '03', 'apr': '04', 'jun': '06',
        'jul': '07', 'aug': '08', 'sep': '09', 'oct': '10', 'nov': '11', 'dec': '12'
    }
    
    # Try to extract year if it exists in the string
    year_match = re.search(r'\b(19|20)\d{2}\b', date_str)
    if year_match:
        year = year_match.group()
        
        # Look for month names
        for month_name, month_num in month_patterns.items():
            if month_name in date_str.lower():
                # Look for day number
                day_match = re.search(r'\b(\d{1,2})(st|nd|rd|th)?\b', date_str)
                if day_match:
                    day = day_match.group(1).zfill(2)
                    return f"{year}-{month_num}-{day}"
                else:
                    # No day found, use 01 as default
                    return f"{year}-{month_num}-01"
    
    # If no recognizable pattern, return None
    # This will be stored as NULL in the database
    return None

def clean_data(input_file, output_file):
    # Load data
    df = pd.read_excel(input_file)
    
    # Create a copy to work with
    df_cleaned = df.copy()
    
    # Remove empty columns
    df_cleaned = df_cleaned.dropna(axis=1, how='all')
    
    # Replace 'n/a' with NaN
    df_cleaned = df_cleaned.replace('n/a', np.nan)
    
    # Normalize titles if column exists
    if 'title' in df_cleaned.columns:
        df_cleaned['title_normalized'] = df_cleaned['title'].apply(normalize_title)
    
    # Clean release dates
    date_cols = ['release_date', 'RelDate']  # Handle both sales and metadata
    for col in date_cols:
        if col in df_cleaned.columns:
            print(f"Cleaning {col} column...")
            df_cleaned[col] = df_cleaned[col].apply(clean_release_date)
            
            # Show some examples of date cleaning
            sample_dates = df_cleaned[col].dropna().head(10)
            if len(sample_dates) > 0:
                print(f"Sample cleaned dates in {col}:")
                for date in sample_dates:
                    print(f"  → {date}")
    
    # Convert financial columns to numeric
    financial_cols = ['international_box_office', 'domestic_box_office',
                     'worldwide_box_office', 'production_budget', 'opening_weekend']
    
    for col in financial_cols:
        if col in df_cleaned.columns:
            df_cleaned[col] = pd.to_numeric(df_cleaned[col], errors='coerce')
    
    # Fill missing text values
    text_cols = ['genre', 'keywords', 'creative_type']
    for col in text_cols:
        if col in df_cleaned.columns:
            df_cleaned[col] = df_cleaned[col].fillna('Unknown')
    
    # Save to CSV
    df_cleaned.to_csv(output_file, index=False)

    
    return df_cleaned

# Usage
if __name__ == "__main__":
    sales_cleaned = clean_data('../data/sales.xlsx', '../cleanedData/sales_cleaned.csv')
    meta_cleaned = clean_data('../data/metaClean43Brightspace.xlsx', '../cleanedData/metadata_cleaned.csv')